In [1]:
import graphlab as gl
import numpy as np
import pandas as pd
import os
from pylab import *

This non-commercial license of GraphLab Create for academic use is assigned to victor.li.zhu@rutgers.edu and will expire on October 06, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1489596109.log


In [4]:
import threading
threading.activeCount()

from ipyparallel import Client
rc = Client()

rc.ids


[0, 1, 2, 3, 4, 5, 6]

In [ ]:
# input parameters
#=================================
classifier_list = ['KNN']
#=================================

window_length_list  = [100, 150, 200, 250, 300]
window_step = 50
feature_type_list   = [1, 2, 3, 12, 13, 23, 123]  # 1: dg, 2: cc, 3: pl, 12: dg+cc, 13: dg+pl, 23: cc+pl, 123: dg+cc+pl
k_list    = range(1,11)

for mouse_id in range(5,6):

    for win_ln_idx in range(len(window_length_list)): # NOTE HERE!!!!! REMOVE "1" AFTER THIS SESSION!!!!!!!!!!!!!
        window_length = window_length_list[win_ln_idx]
        
        ##========================= load features
        # print on screen: progress
        current_progress = ("\n========================================\nLoading features: mouse%02d, window_length%03d ...\n" \
                            % (mouse_id, window_length))
        print current_progress

        # load and organize feature matrix
        comb = lz_load_feature_matrix(mouse_id, window_length, window_step)       
        
        # normalization
        comb_normalization, norm = normalize_features_mice(comb)
        
        for classifier_idx in range(len(classifier_list)):
            classifier = classifier_list[classifier_idx]

            # initiate
            AC = [[0] * len(k_list) for _ in range(len(feature_type_list))] 
            SE = [[0] * len(k_list) for _ in range(len(feature_type_list))] 
            SP = [[0] * len(k_list) for _ in range(len(feature_type_list))] 
            
            for feature_type_idx in range(len(feature_type_list)):
                feature_type = feature_type_list[feature_type_idx]             

                ##========================= classification
                for k_idx in range(len(k_list)):
                    k = k_list[k_idx]                    

                    # print on screen: progress
                    current_progress = ("\n----------------------------------------\nClassifying --> classifier: %s, feature_type%03d, k: %02d ..." \
                                        % (classifier, feature_type, k))
                    print current_progress

                    if classifier == 'LR':
                        AC[feature_type_idx][l2_penalty_idx],SE[feature_type_idx][l2_penalty_idx],SP[feature_type_idx][l2_penalty_idx] = \
                            lz_logistic_AC_SE_SP(comb, feature_type, l2_penalty, l1_penalty)
                    if classifier == 'SVM':
                        AC[feature_type_idx][l2_penalty_idx],SE[feature_type_idx][l2_penalty_idx],SP[feature_type_idx][l2_penalty_idx] = \
                            lz_svm_AC_SE_SP(comb, feature_type, l2_penalty)
                    if classifier == 'RF':
                        AC[feature_type_idx][subSample_idx],SE[feature_type_idx][subSample_idx],SP[feature_type_idx][subSample_idx] = \
                            lz_RF_AC_SE_SP(comb, feature_type, row_subSamp, col_subSamp)
                    if classifier == 'KNN':
                        AC[feature_type_idx][k_idx],SE[feature_type_idx][k_idx],SP[feature_type_idx][k_idx] = \
                            lz_KNN_AC_SE_SP(comb, feature_type, k)

            ##====================== save results
            AC_saveName = ("%s_mouse%02d_window_length%03d_AC.csv" \
                           % (classifier,mouse_id,window_length))
            SE_saveName = ("%s_mouse%02d_window_length%03d_SE.csv" \
                           % (classifier,mouse_id,window_length))
            SP_saveName = ("%s_mouse%02d_window_length%03d_SP.csv" \
                           % (classifier,mouse_id,window_length))
            df_AC = pd.DataFrame(AC)
            df_SE = pd.DataFrame(SE)        
            df_SP = pd.DataFrame(SP)  
            
            os.chdir("/Users/lizhu/Dropbox/projects/calcium/classification_result") 
            df_AC.to_csv(AC_saveName, index = False, header = False)
            df_SE.to_csv(SE_saveName, index = False, header = False)
            df_SP.to_csv(SP_saveName, index = False, header = False)


Loading features: mouse05, window_length100 ...


----------------------------------------
Classifying --> classifier: KNN, feature_type001, k: 01 ...


In [2]:
# load and organize feature matrix
def lz_load_feature_matrix(mouse_id, window_length, window_step):
    fileName = "format4ML_GC6f_emx_0" + str(mouse_id) + "_windowLen" + str(window_length) + "_winStep_0" + str(window_step) + "_v2_threshold_10.csv"
    loadPath = "/Users/lizhu/Dropbox/projects/calcium/format4ML/" + fileName
    comb = gl.SFrame.read_csv(loadPath, delimiter=',',header=False,verbose = False)
    colName_dg = 'degree'
    colName_dg = gl.SArray([colName_dg + repr(i+1) for i in range(30)])
    colName_cc = 'clusteringCoef'
    colName_cc = gl.SArray([colName_cc + repr(i+1) for i in range(30)])
    colName_pl = 'pathlength'
    colName_pl = gl.SArray([colName_pl + repr(i+1) for i in range(30)])
    colName = colName_dg.append(colName_cc.append(colName_pl.append(gl.SArray(['Whisker']))))
    colName = (list(colName))
    dictionary = dict(zip(comb.column_names(), colName))
    comb = comb.rename(dictionary)
    comb = gl.toolkits.cross_validation.shuffle(comb, random_seed=1)
    # comb['Whisker'].show(view = 'Categorical')
    
    return comb

In [3]:
# normalize the features
# feature_matrix is a SFrame, first 90 columns are features, last column is labels.
def normalize_features_mice(comb):
    
    colName = comb.column_names()
    
    # extract feature columns
    comb_np = comb.to_numpy()
    comb_feature = comb_np[:,0:90]
    
    # compute normed features and norms 
    norms = np.linalg.norm(comb_feature, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
    comb_feature_normed = comb_feature / norms
    
    # combine normalized feature vectors and the labels
    comb_normed = np.concatenate((comb_feature_normed,comb_np[:,90:94]),axis = 1)
        
    # convert numpy array to SFrame
    comb_normed = gl.SFrame(comb_normed)
    comb_normed = comb_normed.unpack('X1', '')

    dictionary_norm = dict(zip(comb_normed.column_names(), colName))
    comb_normed = comb_normed.rename(dictionary_norm) # this is the normed version

    return comb_normed, norms # note: norms is array, comb_normed is SFrame

In [4]:
# train and cross-validation
def lz_KNN_AC_SE_SP(data, feature_type, k):
    
    # clearify features
    feature_dg = comb.column_names()[0:30] # feature 1~30: degree, 31~60: clustering coefficient, 60~90: pathlength
    feature_cc = comb.column_names()[30:60]
    feature_pl = comb.column_names()[60:90]
    
    if feature_type == 1:   feature = feature_dg
    if feature_type == 2:   feature = feature_cc
    if feature_type == 3:   feature = feature_pl
    if feature_type == 12:  feature = feature_dg + feature_cc
    if feature_type == 13:  feature = feature_dg + feature_pl
    if feature_type == 23:  feature = feature_cc + feature_pl
    if feature_type == 123: feature = feature_dg + feature_cc + feature_pl
     
    # Kfold
    num_fold = 10
    folds = gl.cross_validation.KFold(comb, num_fold)
    SE = [None] * num_fold
    SP = [None] * num_fold
    AC = [None] * num_fold
    # print specificity
    idx = 0
    for train, valid in folds:
        m = gl.nearest_neighbor_classifier.create(train,
                                          target = 'Whisker',
                                          features = feature,
                                          distance = 'euclidean',
                                          verbose = False)
        confusion_matrix = m.evaluate(valid, 'confusion_matrix', max_neighbors = k)
        confusion_matrix = confusion_matrix.values()[0]
        
        TP, TN, FP, FN = lz_extract_ACC_SE_SP(confusion_matrix)
        
        SP[idx] = float(TN) / (TN + FP)
        SE[idx] = float(TP) / (TP + FN)
        AC[idx] = float(TP+TN) / (TP+TN+FP+FN)
        idx = idx + 1
        
    AC_mn = np.mean(AC)
    AC_sd = np.std(AC)
    SE_mn = np.mean(SE)
    SE_sd = np.std(SE)
    SP_mn = np.mean(SP)
    SP_sd = np.std(SP)
    
    print 'AC_mean = ', AC_mn
    print 'AC_std = ',  AC_sd
    print 'SE_mean = ', SE_mn
    print 'SE_std = ',  SE_sd
    print 'SP_mean = ', SP_mn
    print 'SP_std = ',  SP_sd
    
    return AC_mn, SE_mn, SP_mn

In [5]:
def lz_extract_ACC_SE_SP(confusion_matrix):
    TP = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==1)]
    if np.size(TP) == 0:
        TP = 0
    else:
        TP = TP['count'][0]
    TN = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==0)]
    if np.size(TN) == 0:
        TN = 0
    else:
        TN = TN['count'][0]
    FP = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==1)]
    if np.size(FP) == 0:
        FP = 0
    else:
        FP = FP['count'][0]
    FN = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==0)]
    if np.size(FN) == 0:
        FN = 0
    else:
        FN = FN['count'][0]
    
    return TP, TN, FP, FN

In [ ]:
############### this is the normalized version of the above code

# normalize the features
# feature_matrix is a SFrame, first 90 columns are features, last column is labels.
def normalize_features_mice(comb):
    # extract feature columns
    comb_np = comb.to_numpy()
    comb_feature = comb_np[:,0:90]
    
    # compute normed features and norms 
    norms = np.linalg.norm(comb_feature, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
    comb_feature_normed = comb_feature / norms
    
    # combine normalized feature vectors and the labels
    comb_normed = np.concatenate((comb_feature_normed,comb_np[:,90:91]),axis = 1)
    # convert numpy array to SFrame
    comb_normed = gl.SFrame(comb_normed)
    comb_normed = comb_normed.unpack('X1', '')
    dictionary_norm = dict(zip(comb_normed.column_names(), colName))
    comb_normed = comb_normed.rename(dictionary_norm) # this is the normed version

    return comb_normed, norms # note: norms is array, comb_normed is SFrame

# comb = gl.toolkits.cross_validation.shuffle(comb, random_seed=2)
def lz_logistic(data, feature):
    
    # normalize 
    comb, norms = normalize_features_mice(data)
    comb['Whisker'] = comb['Whisker'].astype(int)

    folds = gl.cross_validation.KFold(comb, 5)
    SE=[None]*5
    SP=[None]*5
    acc=[None]*5
    # print specificity
    idx = 0
    for train, valid in folds:
        m = gl.logistic_classifier.create(train,
                                          target='Whisker',
                                          features=feature,
                                          l2_penalty = 0.05,
                                          validation_set=None,
                                          verbose = False)
        
        # normalize the validation dataset
        valid_np = valid.to_numpy()
        valid_feature = valid_np[:,0:90]

        # use normed from features
        valid_feature_normed = valid_feature / norms

        # combine normalized feature vectors and the labels
        valid_normed = np.concatenate((valid_feature_normed,valid_np[:,90:91]),axis = 1)

        # convert numpy array to SFrame
        valid_normed = gl.SFrame(valid_normed)
        valid_normed = valid_normed.unpack('X1', '')
        dictionary_norm = dict(zip(valid_normed.column_names(), colName))
        valid_normed = valid_normed.rename(dictionary_norm) # this is the normed version
        valid_normed['Whisker'] = valid_normed['Whisker'].astype(int)

        confusion_matrix = m.evaluate(valid, 'confusion_matrix')
        confusion_matrix = confusion_matrix.values()[0]
        TP = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==1)]
        if np.size(TP) == 0:
            TP = 0
        else:
            TP = TP['count'][0]
        TN = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==0)]
        if np.size(TN) == 0:
            TN = 0
        else:
            TN = TN['count'][0]
        FP = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==1)]
        if np.size(FP) == 0:
            FP = 0
        else:
            FP = FP['count'][0]
        FN = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==0)]
        if np.size(FN) == 0:
            FN = 0
        else:
            FN = FN['count'][0]
        SE[idx] = float(TN) / (TN + FP)
        SP[idx] = float(TP) / (TP + FN)
        acc[idx] = float(TP+TN) / (TP+TN+FP+FN)
        idx = idx + 1
    
    print 'acc_mean = ', np.mean(acc)
    print 'acc_std = ', np.std(acc)
    print 'SE_mean = ', np.mean(SE)
    print 'SE_std = ', np.std(SE)
    print 'SP_mean = ', np.mean(SP)
    print 'SP_std = ', np.std(SP)
    
    return acc, SE, SP

# normalize

feature = comb.column_names()[60:90] 

acc,SE,SP = lz_logistic(comb, feature)



In [ ]:
# exploring the data
num_wm = len(comb[comb['Whisker']==1])
num_wn = len(comb[comb['Whisker']==0])
# major class classifier(naive classifier)
major_classifier = float(num_wn)/(num_wm + num_wn)
major_classifier
num_wn

In [ ]:
train_data,test_data = comb.random_split(.8, seed=0)
# train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)
train_data = gl.toolkits.cross_validation.shuffle(train_data, random_seed=1)

In [ ]:
folds = gl.cross_validation.KFold(comb, 5)
folds
params = dict([('target', 'Whisker'), ('features', comb.column_names()[0:90]), ('validation_set', None)])

all_feature_model = gl.cross_validation.cross_val_score(folds,
                                              gl.logistic_classifier.create,
                                              params)

In [ ]:
# print all_feature_model.get_results()
all_feature_model.get_results()

In [ ]:
mylist = sum([range(0,30)] + [range(60,90)],[])
mylist[0:3,5]

In [ ]:
type(colName)

In [ ]:
whisker_model.evaluate(test_data)

In [ ]:
## Weight 
weight = whisker_model['coefficients'].sort('value', ascending=False)
weight['name']

In [ ]:
## 